## Training attention based model

In [2]:
%matplotlib inline

import sys
sys.path.append("../../")
import tensorflow as tf
from train_model import train

from sklearn.linear_model import LinearRegression
from yellowbrick.regressor import ResidualsPlot, PredictionError

import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 4)
rcParams['figure.dpi'] = 100
rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'
rcParams['axes.facecolor'] = '#ffffff'
rcParams['lines.linewidth'] = 2.0

In [1]:
import os
file_path = r"D:\Final_file\ASI-main\output\models\poa\asi_poa_weights.hdf5"
from tensorflow.keras import backend as K
K.clear_session()

if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Removed the file: {file_path}")
else:
    print(f"The file {file_path} does not exist")

Removed the file: D:\Final_file\ASI-main\output\models\poa\asi_poa_weights.hdf5


In [3]:
##BEST POA

hyperparameter={
"num_nearest":60,
"sigma":8,
"geointerpolation": 'asi_multi',
'type_compat_funct_eucli':'identity',
'Num_heads':4,
"learning_rate":0.001,
"batch_size":250,
"num_neuron":60,
"num_layers":3,
"size_embedded":50,
"num_nearest_geo":10,
"num_nearest_eucli":15,
"id_dataset":'poa',
"epochs":300,
"optimier":'adam',
"validation_split":0.1,
"label":'asi_poa',
"early_stopping": False,
'scale_log':True,
"graph_label":'matrix',
}

In [4]:
with tf.device('/device:GPU:0'):
        spatial = train(**hyperparameter)



dataset,\
result,\
fit,\
embedded_train,\
embedded_test,\
predict_regression_train,\
predict_regression_test = spatial()

45/45 [==============================] - 0s 6ms/step - loss: 0.1342 - root_mean_squared_error: 0.1981 - val_loss: 0.1430 - val_root_mean_squared_error: 0.2057

Epoch 00242: val_loss did not improve from 0.13735
Epoch 243/300
45/45 [==============================] - 0s 6ms/step - loss: 0.1319 - root_mean_squared_error: 0.1960 - val_loss: 0.1421 - val_root_mean_squared_error: 0.2040

Epoch 00243: val_loss did not improve from 0.13735
Epoch 244/300
45/45 [==============================] - 0s 6ms/step - loss: 0.1352 - root_mean_squared_error: 0.1984 - val_loss: 0.1414 - val_root_mean_squared_error: 0.2015

Epoch 00244: val_loss did not improve from 0.13735
Epoch 245/300
45/45 [==============================] - 0s 6ms/step - loss: 0.1344 - root_mean_squared_error: 0.1977 - val_loss: 0.1387 - val_root_mean_squared_error: 0.2016

Epoch 00245: val_loss did not improve from 0.13735
Epoch 246/300
45/45 [==============================] - 0s 6ms/step - loss: 0.1346 - root_mean_squared_error: 0.197

In [5]:
print('################# Test ##########################')
print('MALE test:.... {}'.format(result[0]))
print('RMSE test:.... {}'.format(result[1]))
print('MAPE test:.... {}'.format(result[2]))
print('################# Train ##########################')
print('MALE train:.... {}'.format(result[3]))
print('RMSE train:.... {}'.format(result[4]))
print('MAPE train:.... {}'.format(result[5]))

################# Test ##########################
MALE test:.... 59598.24030426564
RMSE test:.... 92829.8777642501
MAPE test:.... 8.868701992753605
################# Train ##########################
MALE train:.... 59598.24030426564
RMSE train:.... 93199.12633577743
MAPE train:.... 8.180629771394047


In [3]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
# Define a list of different values for each hyperparameter
learning_rates = [0.001]
batch_sizes = [250]
num_neurons = [60,50,40]
num_nearest_geo_values = [15,10]
num_nearest_eucli_values = [15,10]
sigma = [2,8,10]


    # Loop through each combination of hyperparameter values
for lr in learning_rates:
        for bs in batch_sizes:
            for nn in num_neurons:
                for num_geo in num_nearest_geo_values:
                    for num_eucli in num_nearest_eucli_values:
                        for sig in sigma:


                            # Define the file path
                            file_path = r"D:\Final_file\ASI-main\output\models\poa\asi_poa_weights.hdf5"
                            from train_model import train
                            from tensorflow.keras import backend as K
                            K.clear_session()
                            # Check if the file exists and remove it
                            if os.path.exists(file_path):
                                os.remove(file_path)
                                print(f"Removed the file: {file_path}")
                            else:
                                print(f"The file {file_path} does not exist")

                            # Update the hyperparameter dictionary with the current combination
                            hyperparameter["learning_rate"] = lr
                            hyperparameter["batch_size"] = bs
                            hyperparameter["num_neuron"] = nn
                            hyperparameter["num_nearest_geo"] = num_geo
                            hyperparameter["num_nearest_eucli"] = num_eucli
                            hyperparameter["sigma"] = num_eucli

                            # Train and evaluate the model
                            with tf.device('/device:GPU:0'):
                                spatial = train(**hyperparameter)

                            dataset,\
                            result,\
                            fit,\
                            embedded_train,\
                            embedded_test,\
                            predict_regression_train,\
                            predict_regression_test = spatial()

                            # Define the output directory
                            output_dir = r'D:\Final_file\ASI-main\notebooks\poa'
                            np.save(os.path.join(output_dir, f"88888TALYit8TRAIN_{lr}_bs_{bs}_nn_{nn}_geo_{num_geo}_eucli_{num_eucli}_{sig}.npy"), embedded_train)
                            np.save(os.path.join(output_dir, f"8888ALYADSmulit8TEST_{lr}_bs_{bs}_nn_{nn}_geo_{num_geo}_eucli_{num_eucli}_{sig}.npy"), embedded_test)
                            # Store the results along with the current hyperparameter values
                            results_dict = {
                                "learning_rate": [lr],
                                "batch_size": [bs],
                                "num_neuron": [nn],
                                "num_nearest_geo": [num_geo],
                                "num_nearest_eucli": [num_eucli],
                                "MALE_test": [result[0]],
                                "RMSE_test": [result[1]],
                                "MAPE_test": [result[2]],
                                "MALE_train": [result[3]],
                                "RMSE_train": [result[4]],
                                "MAPE_train": [result[5]],
                            }

                            # Create a DataFrame from the dictionary
                            results_df = pd.DataFrame(results_dict)

                            # Generate a unique file name for the current combination of hyperparameters
                            file_name = f"8888hdistanceBATCHSIZEmul_ITTEST8heads_{lr}_bs_{bs}_nn_{nn}_geo_{num_geo}_eucli_{num_eucli}_{sig}.csv"

                            # Save the DataFrame as a CSV file
                            results_df.to_csv(file_name, index=False)


Removed the file: D:\Final_file\ASI-main\output\models\poa\asi_poa_weights.hdf5
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Epoch 1/300
45/45 [==============================] - 2s 9ms/step - loss: 6.6153 - root_mean_squared_error: 8.2191 - val_loss: 1.3763 - val_root_mean_squared_error: 1.8304

Epoch 00001: val_loss improved from inf to 1.37630, saving model to d:\Final_file\ASI-main/output/models/poa\asi_poa_weights.hdf5
Epoch 2/300
45/45 [==============================] - 0s 5ms/step - loss: 0.7807 - root_mean_squared_error: 1.0984 - val_loss: 0.4916 - val_root_mean_squared_error: 0.6644

Epoch 00002: val_loss improved from 1.37630 to 0.49163, saving model to d:\Final_file\ASI-main/output/models/poa\asi_poa_weights.hdf5
Epoch 3/300
45/45 [==============================] - 0s 5ms/step - loss: 0.3507 - root_mean_squared_error: 0.4946 - val_loss: 0.2684 - val_roo

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor


## Base models benchmarking 

In [ ]:
X_train ,X_test, y_train , y_test = dataset.X_train,dataset.X_test,dataset.y_train,dataset.y_test

In [ ]:
import numpy as np

# For X_train
unique_X_train, index_X_train, counts_X_train = np.unique(X_train, axis=0, return_index=True, return_counts=True)
duplicated_X_train = len(X_train) - len(unique_X_train)
print(f"Duplicated rows in X_train: {duplicated_X_train}")

# For X_test
unique_X_test, index_X_test, counts_X_test = np.unique(X_test, axis=0, return_index=True, return_counts=True)
duplicated_X_test = len(X_test) - len(unique_X_test)
print(f"Duplicated rows in X_test: {duplicated_X_test}")


In [ ]:

# Define the mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Initialize lists to store test set evaluation metrics for each model
test_mae_list = {}
test_mse_list = {}
test_rmse_list = {}
test_r2_list = {}
test_mape_list = {}

# Define the models and their respective parameter grids
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [5, 10, 15, 20]}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(),
        'params': {'max_depth': [5, 9, 12, 15]}
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {'n_estimators': [50, 100, 150], 'max_depth': [8, 12, 16]}
    },
    'SVM': {
        'model': SVR(),
        'params': {'C': [1, 10, 100], 'gamma': ['scale', 'auto']}
    },
    'LightGBM': {
        'model': LGBMRegressor(),
        'params': {'n_estimators': [1000, 2000], 'learning_rate': [0.01, 0.05, 0.1]}
    },
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0, n_estimators=2000),
        'params': {'depth': [6, 8, 10], 'learning_rate': [0.01, 0.05]}
    },
    'XGBoost': {
        'model': XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=42),
        'params': {'max_depth': [5, 7, 9], 'learning_rate': [0.01, 0.05, 0.1]}
    }
}

# Define the K-fold cross-validator
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# For each model
for name, model_info in models.items():
    test_mae_list[name] = []
    test_mse_list[name] = []
    test_rmse_list[name] = []
    test_r2_list[name] = []
    test_mape_list[name] = []

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Grid Search for hyperparameter tuning
        grid = GridSearchCV(estimator=model_info['model'], param_grid=model_info['params'], cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
        grid_result = grid.fit(X_train_fold, y_train_fold)
        best_model = grid_result.best_estimator_

        # Make predictions on the test set
        y_pred_test = best_model.predict(X_test)

        # Evaluate the model on the test set
        test_mae = mean_absolute_error((y_test), (y_pred_test))
        test_mse = mean_squared_error((y_test), (y_pred_test))
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score((y_test), (y_pred_test))
        test_mape = mean_absolute_percentage_error((y_test), (y_pred_test))

        # Store the test set metrics in the lists
        test_mae_list[name].append(test_mae)
        test_mse_list[name].append(test_mse)
        test_rmse_list[name].append(test_rmse)
        test_r2_list[name].append(test_r2)
        test_mape_list[name].append(test_mape)

    # Calculate the average and best metrics for the test set
    avg_test_mae = np.mean(test_mae_list[name])
    avg_test_mse = np.mean(test_mse_list[name])
    avg_test_rmse = np.mean(test_rmse_list[name])
    avg_test_r2 = np.mean(test_r2_list[name])
    avg_test_mape = np.mean(test_mape_list[name])

    best_test_mae = np.min(test_mae_list[name])
    best_test_mse = np.min(test_mse_list[name])
    best_test_rmse = np.min(test_rmse_list[name])
    best_test_r2 = np.max(test_r2_list[name])
    best_test_mape = np.min(test_mape_list[name])

    print(f"Test Set Evaluation for {name}")
    print(f"Average Test MAE: {avg_test_mae}, Best Test MAE: {best_test_mae}")
    print(f"Average Test MSE: {avg_test_mse}, Best Test MSE: {best_test_mse}")
    print(f"Average Test RMSE: {avg_test_rmse}, Best Test RMSE: {best_test_rmse}")
    print(f"Average Test R2: {avg_test_r2}, Best Test R2: {best_test_r2}")
    print(f"Average Test MAPE: {avg_test_mape}, Best Test MAPE: {best_test_mape}")
    print("\n")


## Testing the embeddings

In [ ]:


X_train ,X_test, y_train , y_test = embedded_train,embedded_test,dataset.y_train,dataset.y_test


# Initialize lists to store test set evaluation metrics for each model
test_mae_list = {}
test_mse_list = {}
test_rmse_list = {}
test_r2_list = {}
test_mape_list = {}

# Define the models and their respective parameter grids
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [5, 10, 15, 20]}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(),
        'params': {'max_depth': [5, 9, 12, 15]}
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {'n_estimators': [50, 100, 150], 'max_depth': [8, 12, 16]}
    },
    'SVM': {
        'model': SVR(),
        'params': {'C': [1, 10, 100], 'gamma': ['scale', 'auto']}
    },
    'LightGBM': {
        'model': LGBMRegressor(),
        'params': {'n_estimators': [1000], 'learning_rate': [0.05, 0.1]}
    },
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0, n_estimators=2000),
        'params': {'depth': [6, 8, 10], 'learning_rate': [0.01, 0.05]}
    },
    'XGBoost': {
        'model': XGBRegressor(n_estimators=2000, learning_rate=0.05, random_state=42),
        'params': {'max_depth': [5, 7, 9], 'learning_rate': [0.01, 0.05, 0.1]}
    }
}

# Define the K-fold cross-validator
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# For each model
for name, model_info in models.items():
    test_mae_list[name] = []
    test_mse_list[name] = []
    test_rmse_list[name] = []
    test_r2_list[name] = []
    test_mape_list[name] = []

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Grid Search for hyperparameter tuning
        grid = GridSearchCV(estimator=model_info['model'], param_grid=model_info['params'], cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
        grid_result = grid.fit(X_train_fold, y_train_fold)
        best_model = grid_result.best_estimator_

        # Make predictions on the test set
        y_pred_test = best_model.predict(X_test)

        # Evaluate the model on the test set
        test_mae = mean_absolute_error((y_test), (y_pred_test))
        test_mse = mean_squared_error((y_test), (y_pred_test))
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score((y_test), (y_pred_test))
        test_mape = mean_absolute_percentage_error((y_test), (y_pred_test))

        # Store the test set metrics in the lists
        test_mae_list[name].append(test_mae)
        test_mse_list[name].append(test_mse)
        test_rmse_list[name].append(test_rmse)
        test_r2_list[name].append(test_r2)
        test_mape_list[name].append(test_mape)

    # Calculate the average and best metrics for the test set
    avg_test_mae = np.mean(test_mae_list[name])
    avg_test_mse = np.mean(test_mse_list[name])
    avg_test_rmse = np.mean(test_rmse_list[name])
    avg_test_r2 = np.mean(test_r2_list[name])
    avg_test_mape = np.mean(test_mape_list[name])

    best_test_mae = np.min(test_mae_list[name])
    best_test_mse = np.min(test_mse_list[name])
    best_test_rmse = np.min(test_rmse_list[name])
    best_test_r2 = np.max(test_r2_list[name])
    best_test_mape = np.min(test_mape_list[name])

    print(f"Test Set Evaluation for {name}")
    print(f"Average Test MAE: {avg_test_mae}, Best Test MAE: {best_test_mae}")
    print(f"Average Test MSE: {avg_test_mse}, Best Test MSE: {best_test_mse}")
    print(f"Average Test RMSE: {avg_test_rmse}, Best Test RMSE: {best_test_rmse}")
    print(f"Average Test R2: {avg_test_r2}, Best Test R2: {best_test_r2}")
    print(f"Average Test MAPE: {avg_test_mape}, Best Test MAPE: {best_test_mape}")
    print("\n")


## Applying pca

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA to embedded_train and embedded_test to reduce embeddings to 12 dimensions
pca = PCA(n_components=6)
embedded_train_pca = pca.fit_transform(embedded_train)
embedded_test_pca = pca.transform(embedded_test)

# Concatenate dataset.X_train with embedded_train_pca and dataset.X_test with embedded_test_pca
X_train_concat = np.concatenate((dataset.X_train, embedded_train_pca), axis=1)
X_test_concat = np.concatenate((dataset.X_test, embedded_test_pca), axis=1)

In [ ]:
X_train ,X_test, y_train , y_test = X_train_concat,X_test_concat,dataset.y_train,dataset.y_test

In [ ]:


# Initialize lists to store test set evaluation metrics for each model
test_mae_list = {}
test_mse_list = {}
test_rmse_list = {}
test_r2_list = {}
test_mape_list = {}

# Define the models and their respective parameter grids
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [5, 10, 15, 20]}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(),
        'params': {'max_depth': [5, 9, 12, 15]}
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {'n_estimators': [50, 100, 150], 'max_depth': [8, 12, 16]}
    },
    'SVM': {
        'model': SVR(),
        'params': {'C': [1, 10, 100], 'gamma': ['scale', 'auto']}
    },
    'LightGBM': {
        'model': LGBMRegressor(),
        'params': {'n_estimators': [1000], 'learning_rate': [0.05, 0.1]}
    },
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0, n_estimators=2000),
        'params': {'depth': [6, 8, 10], 'learning_rate': [0.01, 0.05]}
    },
    'XGBoost': {
        'model': XGBRegressor(n_estimators=2000, learning_rate=0.05, random_state=42),
        'params': {'max_depth': [5, 7, 9], 'learning_rate': [0.01, 0.05, 0.1]}
    }
}

# Define the K-fold cross-validator
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# For each model
for name, model_info in models.items():
    test_mae_list[name] = []
    test_mse_list[name] = []
    test_rmse_list[name] = []
    test_r2_list[name] = []
    test_mape_list[name] = []

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Grid Search for hyperparameter tuning
        grid = GridSearchCV(estimator=model_info['model'], param_grid=model_info['params'], cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
        grid_result = grid.fit(X_train_fold, y_train_fold)
        best_model = grid_result.best_estimator_

        # Make predictions on the test set
        y_pred_test = best_model.predict(X_test)

        # Evaluate the model on the test set
        test_mae = mean_absolute_error((y_test), (y_pred_test))
        test_mse = mean_squared_error((y_test), (y_pred_test))
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score((y_test), (y_pred_test))
        test_mape = mean_absolute_percentage_error((y_test), (y_pred_test))

        # Store the test set metrics in the lists
        test_mae_list[name].append(test_mae)
        test_mse_list[name].append(test_mse)
        test_rmse_list[name].append(test_rmse)
        test_r2_list[name].append(test_r2)
        test_mape_list[name].append(test_mape)

    # Calculate the average and best metrics for the test set
    avg_test_mae = np.mean(test_mae_list[name])
    avg_test_mse = np.mean(test_mse_list[name])
    avg_test_rmse = np.mean(test_rmse_list[name])
    avg_test_r2 = np.mean(test_r2_list[name])
    avg_test_mape = np.mean(test_mape_list[name])

    best_test_mae = np.min(test_mae_list[name])
    best_test_mse = np.min(test_mse_list[name])
    best_test_rmse = np.min(test_rmse_list[name])
    best_test_r2 = np.max(test_r2_list[name])
    best_test_mape = np.min(test_mape_list[name])

    print(f"Test Set Evaluation for {name}")
    print(f"Average Test MAE: {avg_test_mae}, Best Test MAE: {best_test_mae}")
    print(f"Average Test MSE: {avg_test_mse}, Best Test MSE: {best_test_mse}")
    print(f"Average Test RMSE: {avg_test_rmse}, Best Test RMSE: {best_test_rmse}")
    print(f"Average Test R2: {avg_test_r2}, Best Test R2: {best_test_r2}")
    print(f"Average Test MAPE: {avg_test_mape}, Best Test MAPE: {best_test_mape}")
    print("\n")
